# Implementing Random Circuit Sampling on IBM Quantum Devices

## Using IBM Q 16 Melbourne (Automatically Executed, for Data)

Hao Li, Yue Shi, Javad Shabani

New York University

In [1]:
# Import modules to be used in the program

#from qiskit import IBMQ
import numpy as np
from math import *
from qiskit import *
from qiskit.quantum_info import *
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors.standard_errors import pauli_error
from qiskit.providers.aer.noise.errors.standard_errors import thermal_relaxation_error
from qiskit.tools.visualization import plot_histogram
from matplotlib import *
import matplotlib.pyplot as plt
import scipy as sp

qiskit.__qiskit_version__

{'qiskit-terra': '0.9.0',
 'qiskit-ignis': '0.2.0',
 'qiskit-aqua': '0.6.0',
 'qiskit': '0.12.0',
 'qiskit-aer': '0.3.0',
 'qiskit-ibmq-provider': '0.3.2'}

In [2]:
# Enable IBM Quantum Experience account, e.g. IBMQ.enable_account('MY_API_TOKEN')
# Do not enable an account more than once in a session

provider = IBMQ.enable_account('')

In [3]:
# Show the backends accessible

provider.backends()
#backend = provider.get_backend('ibmq_rochester')

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_essex') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_rome') from IBMQ(hub='ibm-q', group='open', project='main')>]

## For each system, choose the number of qubits in the circuits, the range of the number of layers, the number of circuits to test for each number of layers, and the number of experiments you want to run for each circuit, then the program will be automatically executed, and all the data will be meanwhile saved for further usage

### n-qubit system using ibmq_16_melbourne

In [4]:
# Choose the parameters for the series of experients to conduct: 
# (1) the number of qubits in the circuits (qubits2), 
# (2) the range of the number of layers of random circuits ([layers2_min, layers2_max]);
# (3) the number of circuits to test for each number of layers (circs2);
# (4) the number of experiments to run for each circuit (runs2)
# Make sure the numbers are correlated

# **If the backend is very busy, it is suggested to split the range of layers, 
#   increase the step size of layers, or wait for a free time, 
#   so as not to let the program run for too long time

backend2 = provider.get_backend('ibmq_16_melbourne')

# Parameters for the series of experiments to conduct
qubits2 = [10]
layers2_min = [22]
layers2_max = [30]
d_layers2 = 2
circs2 = [15]
runs2 = [8]

# Check if there is anything wrong in the parameters above
# The list of the numbers of qubits in circuits available in this program
qubits_list2 = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
para_error = False
for n in qubits2:
    if n not in qubits_list2:
        print('Circuits not available for: qubits2[%d]' % qi) # Sequence numbers start from 0!
        para_error = True
if len(layers2_min) != len(qubits2):
    print('Number of parameters mismatched with qubits2: layers2_min')
    para_error = True
if len(layers2_max) != len(qubits2):
    print('Number of parameters mismatched with qubits2: layers2_max')
    para_error = True
if len(circs2) != len(qubits2):
    print('Number of parameters mismatched with qubits2: circs2')
    para_error = True
if len(runs2) != len(qubits2):
    print('Number of parameters mismatched with qubits2: runs2')
    para_error = True
if para_error == False:
    print('Parameters confirmed!')

Parameters confirmed!


In [5]:
# Automatically execute the program to perform all the experiments described above and save data

for qi in range(len(qubits2)):
    # The number of qubits used in the circuits, and the dimension of the Hilbert space
    n = qubits2[qi]
    N = 2**n
    for layers2 in range(layers2_min[qi], layers2_max[qi] + 1, d_layers2):
        for n_circ2 in range(1, circs2[qi] + 1):
            # n qubit circuits initialization (using ibmq_16_melbourne)
            # The number of qubits registered
            if n in range(4, 12):
                n_reg2 = 13
            elif n in range(12, 14):
                n_reg2 = 14
            else:
                n_reg2 = 15
            # Vertices of the rectangular/trapezoid of the qubits to be used
            if n in [5, 7, 9, 11, 12]:
                vert0 = 1
            elif n in [4, 6, 8, 10]:
                vert0 = 2
            else:
                vert0 = 0
            if n in range(4, 12):
                vert1 = int(n / 2) + 1
                vert2 = 14 - vert1
            if n in range(12, 16):
                vert1 = 6
                if n == 15:
                    vert2 = 7
                else:
                    vert2 = 8
            vert3 = n_reg2 - 1
            # List of qubits used in the circuits
            q2_list = []
            for i in range(vert0, vert1 + 1):
                q2_list.append(i)
            for i in range(vert2, vert3 + 1):
                q2_list.append(i)
            q2 = QuantumRegister(n_reg2)
            c2 = ClassicalRegister(n_reg2)
            circ2 = QuantumCircuit(q2, c2)
            # Generate n-qubit random circuits of a number of layers
            for k in range(0, layers2):
                # For every qubit, add a random unitary u3 gate
                for i in q2_list:    
                    u = quantum_info.random_unitary(2).data
                    Theta = 2 * np.arccos(abs(u[0][0]))
                    Phi = np.angle(u[1][0] / (u[0][0] * np.sqrt(1 - abs(u[0][0])**2)))
                    Lambda =  np.angle(- u[0][1] / (u[0][0] * np.sqrt(1 - abs(u[0][0])**2)))
                    Phi += 2 * np.pi * int(- np.sign(Phi) + 0.5)
                    Lambda += 2 * np.pi * int(- np.sign(Lambda) + 0.5)
                    circ2.u3(Theta, Phi, Lambda, q2[i])
                # Apply CNOT on several random couples of (neighbor) qubits that do not overlap
                used = []
                ranlist = []
                for l in range(n - 1):
                    ranlist.append(np.random.randint(0, n))
                for l in ranlist:
                    # In the rectangular/trapezoidal map of the qubits, except for the vertices, 
                    # apply equal probabilities for the three connected neighbors:
                    # 1/3 for vertical neighbor, and 2/3 for horizontal neighbors, 
                    # with 1/3 for each side. For the four vertices, 
                    # only one horizontal neighbor is available, and for odd n, 
                    # the single connected qubit also does not have a vertical neighbor
                    i = q2_list[l]
                    m = np.random.randint(0, 4)
                    # 1/3 probability for vertical neighbor if available
                    if m == 3:
                        if (14 - i) in q2_list:
                            j = 14 - i
                    # For vertices, choose the only horizontal neighbor
                    elif i == vert0 or i == vert2:
                        j = i + 1
                    elif i == vert1 or i == vert3:
                        j = i - 1
                    # 2/3 probabilities for horizontal neighbor, half for each side
                    else:
                        j = i + (- 1)**m
                    # Exclude the couples that overlap with the used couples in each layer
                    if (j not in used) and (i not in used):
                        if i != j: # Exclude the i = j = 7 vertical case
                            # Directions already included in the orders of couples
                            circ2.cx(q2[i], q2[j])
                            used.append(j)
                            used.append(i)
            
            # Run it on statevector_simulator to work out the state amplitudes
            circuit = circ2
            simulatorI = Aer.get_backend('statevector_simulator')
            job2 = execute(circuit, simulatorI)
            result2 = job2.result()
            jobID2 = job2.job_id()
            # Get the statevector from result2()
            statevector2 = result2.get_statevector(circuit)
            # Get the probabilities (state2 with data cleaning order) and sort(state2_sort)
            state2 = []
            stvec2 = []
            for i in range(0, 2**n_reg2):
                if abs(statevector2[i]) != 0:
                    state2.append(np.abs(statevector2[i])**2)
                    stvec2.append(statevector2[i])
            state2 = np.array(state2)
            state2_sort = - np.sort(- state2)
            # Save the statevector simulation results to a file (in data cleaning order)
            stvec2 = np.array(stvec2)
            data2 = np.column_stack((stvec2, state2))
            filename = "%d-qubit_%s_stvec_L%d_C%d_%s.csv" % (n, backend2, layers2, n_circ2, 
                                                             jobID2)
            np.savetxt(filename, data2, delimiter = ',')
            # Ideal (discrete) Porter-Thomas distribution
            f = []
            for i in range(0, N):
                if i == 0:
                    f.append((log(N / (i + 1)) + 1) / N)
                else:
                    f.append((log(N / (i + 1)) + 1 - i * log ((i + 1) / i)) / N)
            # XEB fidelities of statevector simulation (original and simplified)
            S_inc_ptd = 0
            S_exp_ptd = 0
            S_ptd = 0
            for i in range(0, N):
                S_inc_ptd += - log(f[i]) / N
                S_exp_ptd += - log(f[i]) * state2_sort[i]
                S_ptd += - log(f[i]) * f[i]
            F_XEB_15_stvec = (S_inc_ptd - S_exp_ptd) / (S_inc_ptd - S_ptd)
            F_XEB_sim_15_stvec = log(N) + np.euler_gamma - S_exp_ptd
            # Linear XEB (LXEB) fidelities of statevecror simulation (original and simplified)
            SL_inc_ptd = 0
            SL_exp_ptd = 0
            SL_ptd = 0
            for i in range(0, N):
                SL_inc_ptd += (1 - f[i]) / N
                SL_exp_ptd += (1 - f[i]) * state2_sort[i]
                SL_ptd += (1 - f[i]) * f[i]
            F_LXEB_15_stvec = (SL_inc_ptd - SL_exp_ptd) / (SL_inc_ptd - SL_ptd)
            F_LXEB_sim_15_stvec = 2**n * (1 - SL_exp_ptd) - 1
            
            # Save the circuit (no measurement) for statevector simulation as an OpenQasm file
            circ2.qasm()
            file = open("%d_qubit_circuit_%s_no_mea_L%d_C%d.qasm" % 
                        (n, backend2, layers2, n_circ2), 'w')
            file.write(circ2.qasm())
            file.close()
            
            # Apply measurement gates to the circuit, so that you can run it on IBM device
            for i in q2_list:
                circ2.measure(q2[i], c2[i])
            
            # Save the circuit as an OpenQasm file
            circ2.qasm()
            file = open("%d_qubit_circuit_%s_L%d_C%d.qasm" % 
                        (n, backend2, layers2, n_circ2), 'w')
            file.write(circ2.qasm())
            file.close()
        
            # Run it on actual IBM system (ibmq_16_melbourne)
            counts_4_total = []
            prob_4_total = []
            jobID_4 = []
            for run in range(1, runs2[qi] + 1):
                shots_4 = 8192
                backend2 = provider.get_backend('ibmq_16_melbourne')
                job_4 = execute(circ2, backend2, shots = shots_4)
                result_4 = job_4.result().get_counts(circ2)
                jobID_4.append(job_4.job_id())
                # Data cleaning states(st_4), experimental counts(counts_4) & 
                # probabilities(prob_4)
                st = []
                counts = []
                prob = []
                for i in range(0, 2**n_reg2):
                    s = '0' * n_reg2
                    bin_i = np.binary_repr(i)
                    s_i = s[0: n_reg2 - len(bin_i)] + bin_i
                    st.append(s_i)
                    prob.append(np.abs(statevector2[i])**2)
                    try:
                        counts.append(result_4[s_i])
                    except:
                        counts.append(0)
                st_4 = []
                counts_4 = []
                prob_4 = []
                for i in range(len(prob)):
                    if prob[i] != 0:
                        st_4.append(st[i])
                        counts_4.append(counts[i])
                        prob_4.append(counts[i] / shots_4) 
                if run == 1:
                    counts_4_total = np.array(counts_4)
                else:
                    counts_4_total = np.array(counts_4_total) + np.array(counts_4)
                # Save the experimental results to files separately
                data_4 = np.column_stack((counts_4, prob_4))
                filename = "%d-qubit_%s_L%d_C%d_R%d_%s.csv" % (n, backend2, layers2, n_circ2, 
                                                               run, jobID_4[run - 1]) 
                np.savetxt(filename, data_4, delimiter = ',')
            # Save the total counts and probabilities of all these results
            prob_4_total = np.array(counts_4_total) / (runs2[qi] * shots_4)
            data_4_total = np.column_stack((counts_4_total, prob_4_total))
            filename = "%d_qubit_%s_L%d_C%d_R%d_total.csv" % (n, backend2, layers2, n_circ2, 
                                                              runs2[qi])
            np.savetxt(filename, data_4_total, delimiter = ',')
            # Sort the statevector simulation and experimental results by statevector simulation
            prob_4_total_sort = []
            state2_sort = []
            prob_4_total_sort = np.array(prob_4_total)
            state2_sort = - np.array(state2)
            state2_sort, prob_4_total_sort = zip(*sorted(zip(state2_sort, prob_4_total_sort)))
            state2_sort = - np.array(state2_sort)
            # XEB fidelities of experimental results (original and simplified)
            S_inc_exp = 0
            S_mea_exp = 0
            S_exp = 0
            for i in range(0, N):
                S_inc_exp += - log(state2[i]) / N
                S_mea_exp += - log(state2[i]) * prob_4_total[i]
                S_exp += - log(state2[i]) * state2[i]
            F_XEB_15 = (S_inc_exp - S_mea_exp) / (S_inc_exp - S_exp)
            F_XEB_sim_15 = log(N) + np.euler_gamma - S_mea_exp
            # LXEB fidelities of experimental results (original and simplified)
            SL_inc_exp = 0
            SL_mea_exp = 0
            SL_exp = 0
            for i in range(0, N):
                SL_inc_exp += (1 - state2[i]) / N
                SL_mea_exp += (1 - state2[i]) * prob_4_total[i]
                SL_exp += (1 - state2[i]) * state2[i]
            F_LXEB_15 = (SL_inc_exp - SL_mea_exp) / (SL_inc_exp - SL_exp)
            F_LXEB_sim_15 = N * (1 - SL_mea_exp) - 1
            # k-th order XEB fidelities (k = 2, 3, 4, 5, 6, and inverse) (original)
            F_kXEB_15 = []
            F_kXEB_sim_15 = []
            for k in range(2, 7):
                Sk_inc_exp = 0
                Sk_mea_exp = 0
                Sk_exp = 0
                Pk_mea_exp = 0
                for i in range(0, N):
                    Sk_inc_exp += (1 - state2[i]**k) / N
                    Sk_mea_exp += (1 - state2[i]**k) * prob_4_total[i]
                    Sk_exp += (1 - state2[i]**k) * state2[i]
                    Pk_mea_exp += (state2[i]**k) * prob_4_total[i]
                F_kXEB_15.append((Sk_inc_exp - Sk_mea_exp) / (Sk_inc_exp - Sk_exp))
                F_kXEB_sim_15.append((Pk_mea_exp * N**k / gamma(k + 1) - 1) / k)
            F_1kXEB_15 = []
            F_1kXEB_sim_15 = []
            for k in range(2, 7):
                S1k_inc_exp = 0
                S1k_mea_exp = 0
                S1k_exp = 0
                P1k_mea_exp = 0
                for i in range(0, N):
                    S1k_inc_exp += (1 - state2[i]**(1 / k)) / N
                    S1k_mea_exp += (1 - state2[i]**(1 / k)) * prob_4_total[i]
                    S1k_exp += (1 - state2[i]**(1 / k)) * state2[i]
                    P1k_mea_exp += (state2[i]**(1 / k)) * prob_4_total[i]
                F_1kXEB_15.append((S1k_inc_exp - S1k_mea_exp) / (S1k_inc_exp - S1k_exp))
                F_1kXEB_sim_15.append((P1k_mea_exp * N**(1 / k) / gamma(1 / k + 1) - 1) * k)
            
            # Build noise model for noise simulation
            basis_gates = ['u1', 'u2', 'u3', 'cx', 'id']
            noise_model2 = NoiseModel(basis_gates)
            p_ui = 0.0015 # Single qubit unitary gate Pauli error rate
            p_id = 0.0015 # Identical gate Pauli error rate
            p_cx0 = 0.01 # CNOT Pauli error rate 'XX'
            p_cx1 = 0.03 # CNOT Pauli error rate 'XI'
            p_cx2 = 0.03 # CNOT Pauli error rate 'IX'
            p1given0 = 0.07 # Readout error to get |1> given |0>
            p0given1 = 0.07 # Readout error to get |0> given |1>
            T1 = 56.1 # Qubit population thermal relaxation time (in microseconds)
            T2 = 56.8 # Qubit phase thermal relaxation time (in microseconds)
            t1 = 0.05 # Single qubit gate time (in microseconds)
            t2 = 0.1 # Two qubit gate time (in microseconds)
            esp = 0.001 # Excited state population at equilibrium (default: |0>)
            p_mea = 0.01 # Measurement Pauli error rate
            p_res = 0.01 # Reset Pauli error rate
            error_ui = pauli_error([('X', p_ui), ('I', 1 - p_ui)])
            error_id = pauli_error([('X', p_id), ('I', 1 - p_id)])
            error_cx = pauli_error([('XX', p_cx0), ('XI', p_cx1), 
                                    ('IX', p_cx2), ('II', 1 - p_cx0 - p_cx1 - p_cx2)])
            error_ro = [[1 - p1given0, p1given0], [p0given1, 1 - p0given1]]
            error_tr1 = thermal_relaxation_error(T1, T2, t1, esp)
            error_tr2 = (thermal_relaxation_error(T1, T2, t2, esp).
                         expand(thermal_relaxation_error(T1, T2, t2, esp)))
            error_mea = pauli_error([('X', p_mea), ('I', 1- p_mea)])
            error_res = pauli_error([('X', p_res), ('I', 1 - p_res)])
            # All qubit error for each gate can only be applied once
            #noise_model2.add_all_qubit_quantum_error(error_ui, ['u1', 'u2', 'u3'])
            #noise_model2.add_all_qubit_quantum_error(error_id, 'id')
            noise_model2.add_all_qubit_quantum_error(error_cx, 'cx')
            noise_model2.add_all_qubit_readout_error(error_ro)
            noise_model2.add_all_qubit_quantum_error(error_tr1, ['u1', 'u2', 'u3'])
            noise_model2.add_all_qubit_quantum_error(error_tr1, 'id')
            #noise_model2.add_all_qubit_quantum_error(error_tr2, 'cx')
            noise_model2.add_all_qubit_quantum_error(error_mea, 'measure')
            noise_model2.add_all_qubit_quantum_error(error_res, 'reset')
            #noise_model2.add_quantum_error(error, 'reset', [1])
            #noise_model2.add_nonlocal_quantum_error(error, 'reset', [0], [1])
            # Get coupling map from backend
            backend2 = provider.get_backend('ibmq_16_melbourne')
            coupling_map = backend2.configuration().coupling_map
            # Run it on Qasm simulator to simulate results with noise on classical computer
            shots_6 = 2**20
            simulatorII = Aer.get_backend('qasm_simulator')
            job_6 = execute(circ2, simulatorII, noise_model = noise_model2, 
                            coupling_map = coupling_map, shots = shots_6)
            result_6 = job_6.result().get_counts(circ2)
            jobID_6 = job_6.job_id()
            # Data cleaning states(st_6), simulated counts(counts_6) & probabilities(prob_6)
            st = []
            counts = []
            prob = []
            for i in range(0, 2**n_reg2):
                s = '0' * n_reg2
                bin_i = np.binary_repr(i)
                s_i = s[0: n_reg2 - len(bin_i)] + bin_i
                st.append(s_i)
                prob.append(np.abs(statevector2[i])**2)
                try:
                    counts.append(result_6[s_i])
                except:
                    counts.append(0) 
            st_6 = []
            counts_6 = []
            prob_6 = []
            for i in range(len(prob)):
                if prob[i] != 0:
                    st_6.append(st[i])
                    counts_6.append(counts[i])
                    prob_6.append(counts[i] / shots_6)
            # Save the Qasm noise simulation results to a file (in data cleaning order)
            data_6 = np.column_stack((counts_6, prob_6))
            filename = "%d-qubit_%s_qasm_noise_L%d_C%d_%s.csv" % (n, backend2, layers2, n_circ2, 
                                                                  jobID_6)
            np.savetxt(filename, data_6, delimiter = ',')
            # Sort the statevector and noise simulation results by statevector simulation results
            prob_6_sort = []
            state2_sort = []
            prob_6_sort = np.array(prob_6)
            state2_sort = - np.array(state2)
            state2_sort, prob_6_sort = zip(*sorted(zip(state2_sort, prob_6_sort)))
            state2_sort = - np.array(state2_sort)
            # XEB fidelities of Qasm noise simulation (original and simplified) 
            S_inc_ptd = 0
            S_exp_ptd = 0
            S_ptd = 0
            for i in range(0, N):
                S_inc_ptd += - log(f[i]) / N
                S_exp_ptd += - log(f[i]) * prob_6_sort[i]
                S_ptd += - log(f[i]) * f[i]
            F_XEB_15_qasm_noise = (S_inc_ptd - S_exp_ptd) / (S_inc_ptd - S_ptd)
            F_XEB_sim_15_qasm_noise = log(N) + np.euler_gamma - S_exp_ptd
            # Linear XEB fidelities of Qasm noise simulation (original and simplified)
            SL_inc_exp = 0
            SL_mea_exp = 0
            SL_exp = 0
            for i in range(0, N):
                SL_inc_exp += (1 - f[i]) / N
                SL_mea_exp += (1 - f[i]) * prob_6_sort[i]
                SL_exp += (1 - f[i]) * f[i]
            F_LXEB_15_qasm_noise = (SL_inc_exp - SL_mea_exp) / (SL_inc_exp - SL_exp)
            F_LXEB_sim_15_qasm_noise = N * (1 - SL_mea_exp) - 1
            # Save all the calculated fidelities ((noise) simulation, and experiment)
            filename = "%d-qubit_%s_fidelities_L%d_C%d.csv" % (n, backend2, layers2, n_circ2)
            file2 = open(filename, "w")
            file2.write("%s\n" % backend2.status())
            file2.write("%s\n" % simulatorI.status())
            file2.write("%s\n" % simulatorII.status())
            file2.write("Number of qubits in the circuit, %d\n" % n)
            file2.write("Number of layers of circuits, %d\n" % layers2)
            file2.write("Circuit sequence number, %d\n" % n_circ2)
            file2.write("Depth of the circuit, %d\n" % circ2.depth())
            file2.write("Number of runs (8192 times each), %d\n\n" % runs2[qi])
            file2.write("Job ID of simulations and experiments\n")
            file2.write("Statevector simulation, %s\n" % jobID2)
            file2.write("Qasm noise simulation, %s\n" % jobID_6)
            for run in range(1, runs2[qi] + 1):
                file2.write("%s experiment %d, %s\n" % (backend2, run, jobID_4[run - 1]))
            file2.write("\n")
            file2.write("Fidelities (original and simplified) of simulations and experiment,")
            file2.write("Origial idelities, Simplified fidelities, k for k'th order fidelities\n")
            file2.write("F_XEB(_sim)_statevector, %.17f, %.17f\n" % 
                        (F_XEB_15_stvec, F_XEB_sim_15_stvec))
            file2.write("F_LXEB(_sim)_statevector, %.17f, %.17f\n" % 
                        (F_LXEB_15_stvec, F_LXEB_sim_15_stvec))
            file2.write("F_XEB(_sim)_qasm_noise, %.17f, %.17f\n" % 
                        (F_XEB_15_qasm_noise, F_XEB_sim_15_qasm_noise))
            file2.write("F_LXEB(_sim)_qasm_noise, %.17f, %.17f\n" % 
                        (F_LXEB_15_qasm_noise, F_LXEB_sim_15_qasm_noise))
            file2.write("F_XEB(_sim), %.17f, %.17f\n" % (F_XEB_15, F_XEB_sim_15))
            file2.write("F_LXEB(_sim), %.17f, %.17f, 1\n" % (F_LXEB_15, F_LXEB_sim_15))
            for k in range (2, 7):
                file2.write("F_kXEB(_sim), %.17f, %.17f, %d\n" % 
                            (F_kXEB_15[k - 2], F_kXEB_sim_15[k - 2], k))
            for k in range (2, 7):
                file2.write("F_kXEB(_sim), %.17f, %.17f, %.17f, 1/%d\n" % 
                            (F_1kXEB_15[k - 2], F_1kXEB_sim_15[k - 2], (1 / k), k))
            file2.close()
            
            print("Qubits: %d, Layers: %d, Circuits: %d" % (n, layers2, n_circ2))

print("Finished!")

Qubits: 10, Layers: 22, Circuits: 1
Qubits: 10, Layers: 22, Circuits: 2
Qubits: 10, Layers: 22, Circuits: 3
Qubits: 10, Layers: 22, Circuits: 4
Qubits: 10, Layers: 22, Circuits: 5
Qubits: 10, Layers: 22, Circuits: 6
Qubits: 10, Layers: 22, Circuits: 7
Qubits: 10, Layers: 22, Circuits: 8
Qubits: 10, Layers: 22, Circuits: 9
Qubits: 10, Layers: 22, Circuits: 10
Qubits: 10, Layers: 22, Circuits: 11
Qubits: 10, Layers: 22, Circuits: 12
Qubits: 10, Layers: 22, Circuits: 13
Qubits: 10, Layers: 22, Circuits: 14
Qubits: 10, Layers: 22, Circuits: 15
Qubits: 10, Layers: 24, Circuits: 1
Qubits: 10, Layers: 24, Circuits: 2
Qubits: 10, Layers: 24, Circuits: 3
Qubits: 10, Layers: 24, Circuits: 4
Qubits: 10, Layers: 24, Circuits: 5
Qubits: 10, Layers: 24, Circuits: 6
Qubits: 10, Layers: 24, Circuits: 7
Qubits: 10, Layers: 24, Circuits: 8
Qubits: 10, Layers: 24, Circuits: 9
Qubits: 10, Layers: 24, Circuits: 10
Qubits: 10, Layers: 24, Circuits: 11
Qubits: 10, Layers: 24, Circuits: 12
Qubits: 10, Layers:

Unexpected exception in keepalive ping task
Traceback (most recent call last):
  File "C:\Users\29443\Anaconda3\lib\site-packages\websockets\protocol.py", line 984, in keepalive_ping
    ping_waiter = yield from self.ping()
  File "C:\Users\29443\Anaconda3\lib\site-packages\websockets\protocol.py", line 583, in ping
    yield from self.ensure_open()
  File "C:\Users\29443\Anaconda3\lib\site-packages\websockets\protocol.py", line 658, in ensure_open
    ) from self.transfer_data_exc
websockets.exceptions.ConnectionClosed: WebSocket connection is closed: code = 4002 (private use), no reason


Qubits: 10, Layers: 26, Circuits: 9
Qubits: 10, Layers: 26, Circuits: 10
Qubits: 10, Layers: 26, Circuits: 11
Qubits: 10, Layers: 26, Circuits: 12
Qubits: 10, Layers: 26, Circuits: 13


Unexpected exception in keepalive ping task
Traceback (most recent call last):
  File "C:\Users\29443\Anaconda3\lib\site-packages\websockets\protocol.py", line 984, in keepalive_ping
    ping_waiter = yield from self.ping()
  File "C:\Users\29443\Anaconda3\lib\site-packages\websockets\protocol.py", line 583, in ping
    yield from self.ensure_open()
  File "C:\Users\29443\Anaconda3\lib\site-packages\websockets\protocol.py", line 658, in ensure_open
    ) from self.transfer_data_exc
websockets.exceptions.ConnectionClosed: WebSocket connection is closed: code = 4002 (private use), no reason


Qubits: 10, Layers: 26, Circuits: 14
Qubits: 10, Layers: 26, Circuits: 15
Qubits: 10, Layers: 28, Circuits: 1
Qubits: 10, Layers: 28, Circuits: 2
Qubits: 10, Layers: 28, Circuits: 3
Qubits: 10, Layers: 28, Circuits: 4
Qubits: 10, Layers: 28, Circuits: 5
Qubits: 10, Layers: 28, Circuits: 6
Qubits: 10, Layers: 28, Circuits: 7
Qubits: 10, Layers: 28, Circuits: 8
Qubits: 10, Layers: 28, Circuits: 9
Qubits: 10, Layers: 28, Circuits: 10
Qubits: 10, Layers: 28, Circuits: 11
Qubits: 10, Layers: 28, Circuits: 12
Qubits: 10, Layers: 28, Circuits: 13
Qubits: 10, Layers: 28, Circuits: 14
Qubits: 10, Layers: 28, Circuits: 15
Qubits: 10, Layers: 30, Circuits: 1


Error checking job status using websocket, retrying using HTTP.


Qubits: 10, Layers: 30, Circuits: 2
Qubits: 10, Layers: 30, Circuits: 3
Qubits: 10, Layers: 30, Circuits: 4
Qubits: 10, Layers: 30, Circuits: 5
Qubits: 10, Layers: 30, Circuits: 6
Qubits: 10, Layers: 30, Circuits: 7
Qubits: 10, Layers: 30, Circuits: 8
Qubits: 10, Layers: 30, Circuits: 9
Qubits: 10, Layers: 30, Circuits: 10
Qubits: 10, Layers: 30, Circuits: 11
Qubits: 10, Layers: 30, Circuits: 12
Qubits: 10, Layers: 30, Circuits: 13
Qubits: 10, Layers: 30, Circuits: 14
Qubits: 10, Layers: 30, Circuits: 15
Finished!
